# Match most similar competency

## Existing competencies

In [20]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [21]:
competencys = ['Creativity', 'Employee loyalty', 'Reliability', 'Leadership', 'Passion', 'People Skills', 'Proactivity', 'Team Work']
for i in range(8):
    print "{}:{}".format(i, competencys[i])


0:Creativity
1:Employee loyalty
2:Reliability
3:Leadership
4:Passion
5:People Skills
6:Proactivity
7:Team Work


In [22]:
descriptions = ['Ability to think in a divergent way, and generate novel ideas.',
                'Be committed to the success of the organization and believe that working for this organization is their best option.',
                'Ability to performing to perform when needed, finishing projects and meeting deadlines.', 
                'Ability to engage and facilitate others to create a better result.', 
                'Desire to provide long-term commitment to their organization, demonstrate peak performance, and maintain increased tenure with the organization. With the intense desire or enthusiasm for the work.', 
                'Ability to establish social connections and understand the mindset of others.', 
                'Take the initiative and responsibility in improving business, rather than looking for causes in outside circumstances or other people.',
                'Ability to adapt to the needs of a group of people, while supporting the work of others.']

## load dictionary and corpus

In [23]:
from gensim import corpora, similarities, models

dictionary = corpora.Dictionary.load_from_text('/mnt/local/var/seedlink/data/dictionary/dict_en.txt')
print(dictionary)

Dictionary(50920 unique tokens: [u'sucess', u"cake'", u'mid-week', u'stallions', u'sonja']...)


In [24]:
import codecs
import json
path = '/mnt/local/var/seedlink/data/nltk_data/corpora/prepcorp/en_cc.json'
with codecs.open(path, 'r', 'utf8') as fp:
    jdata = fp.read()
data = json.loads(jdata)
corpus = data

In [25]:
for i in range(10):
    words =  [dictionary[t[0]] for t in corpus[i]]
     #print corpus[i]
    #print ' '.join(words)

## New competency

In [26]:
new_comp = """
Charisma	Able to establish social connections and a broad network
Confidence	Acts on his/her own initiative with confidence
Creativity	Seeks change and performs creatively
Entrepreneurial	Keeps aware of both organizational issues and market opportunities
Goal Orientated	Works systematically and drives the project to its goals
Goal Orientation	Work systematically and drive the project to the target goals
Growth Potential	Growth Potential
Integrity	Acts with integrity and shows social responsibility
Intellect	Flexible application of knowledge to differing contexts
Job Performance	Job Performance
Leadership	Identifies talents, empowers and motivates other
Logic	Logical thinking and rational analysis
Organisation	Sets objectives, balances resources and time, monitors progress
Organization	Sets objectives, balances resources and time, monitors progress
Performance	Job Performance
Proactive	Proactively deals with ambiguity
Process driven	Demonstrate commitment, accountablity, follow policies and procedures of the organizations
Resilient	Coping and resilient under stressful environment
Self motivated	Work enthusiastically for the achievement of self-development
Strategic Competency	Strategic Competency
Strategy	Foresight for potential problems and comes up with appropriate solutions
Teamwork	Adapts to the team; Supports others
"""

lines = new_comp.lower().strip().split('\n')
new_comp_dict = {splitted[0]:splitted[1] for splitted in [line.split('\t') for line in lines]}
new_comp_dict

{'charisma': 'able to establish social connections and a broad network',
 'confidence': 'acts on his/her own initiative with confidence',
 'creativity': 'seeks change and performs creatively',
 'entrepreneurial': 'keeps aware of both organizational issues and market opportunities',
 'goal orientated': 'works systematically and drives the project to its goals',
 'goal orientation': 'work systematically and drive the project to the target goals',
 'growth potential': 'growth potential',
 'integrity': 'acts with integrity and shows social responsibility',
 'intellect': 'flexible application of knowledge to differing contexts',
 'job performance': 'job performance',
 'leadership': 'identifies talents, empowers and motivates other',
 'logic': 'logical thinking and rational analysis',
 'organisation': 'sets objectives, balances resources and time, monitors progress',
 'organization': 'sets objectives, balances resources and time, monitors progress',
 'performance': 'job performance',
 'proac

## Tfidf

In [27]:
tfidf = models.TfidfModel(corpus)

In [28]:
desc_bow = [dictionary.doc2bow(desc.lower().split()) for desc in descriptions]
desc_tfidf = tfidf[desc_bow]

## LDA

In [29]:
# lda = models.LdaModel(tfidf[corpus], num_topics=100, id2word=dictionary)

In [30]:
#lda.save('/tmp/competency-100-orig.lda')
lda = models.LdaModel.load('/tmp/competency-100-orig.lda')

In [31]:
print lda

LdaModel(num_terms=50920, num_topics=100, decay=0.5, chunksize=2000)


In [33]:
lda.update(desc_tfidf)

In [34]:
lda.print_topics(5)

[(82,
  u'0.025*"rather" + 0.017*"house" + 0.016*"committed" + 0.016*"big" + 0.014*"somewhat" + 0.013*"improving" + 0.011*"fascinating" + 0.010*"imagination" + 0.010*"screen" + 0.010*"tried"'),
 (9,
  u'0.036*"black" + 0.030*"side" + 0.028*"woman" + 0.025*"guy" + 0.025*"perform" + 0.024*"cop" + 0.021*"features" + 0.020*"white" + 0.019*"bunch" + 0.018*"old"'),
 (61,
  u'0.012*"to" + 0.011*"the" + 0.011*"looked" + 0.011*"performing" + 0.011*"force" + 0.010*"that" + 0.010*"a" + 0.010*"of" + 0.009*"and" + 0.009*"waiting"'),
 (19,
  u'0.026*"desire" + 0.020*"violence" + 0.019*"money" + 0.018*"contains" + 0.016*"e" + 0.016*"desperate" + 0.014*"trip" + 0.013*"empty" + 0.013*"suffering" + 0.011*"families"'),
 (51,
  u'0.021*"wants" + 0.015*"minute" + 0.012*"to" + 0.011*"a" + 0.010*"ride" + 0.010*"minor" + 0.009*"of" + 0.009*"the" + 0.009*"as" + 0.009*"and"')]

In [35]:
index = similarities.MatrixSimilarity(lda[desc_tfidf]) # lda model

In [36]:
query = 'Able to establish social connections and a broad network'.lower()
query_bow = dictionary.doc2bow(query.split())
print query_bow
query_lda = lda[tfidf[query_bow]]
query_lda
sims = index[query_lda]


[(3, 1), (4, 1), (14, 1), (1588, 1), (3670, 1), (6227, 1), (9190, 1), (12257, 1), (15255, 1)]


In [37]:
competencys[3]

'Leadership'

In [38]:
# query by competency name
for comp in new_comp_dict:
    query = comp.lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lda = lda[tfidf[query_bow]]

    sims = index[query_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Passion             	0.17106346786
growth potential         	Reliability         	0.352613568306
integrity                	Creativity          	0.0
intellect                	Proactivity         	0.135641306639
confidence               	Creativity          	0.0
goal orientated          	Employee loyalty    	0.407724529505
job performance          	Creativity          	0.0
organisation             	Creativity          	0.732443153858
creativity               	Creativity          	0.0
self motivated           	Proactivity         	0.708008766174
strategy                 	Creativity          	0.0
charisma                 	Passion             	0.290378719568
teamwork                 	Creativity          	0.0
resilient                	Creativity          	0.0
performance              	Creativity          	0.0
goal orientation         	Creativity          	0.38825944066
leadership               	Creativity          	0.0
entrepreneurial          	Creativity          	

In [39]:
# query by competency desc
for comp in new_comp_dict:
    query = new_comp_dict[comp].lower()
    query_bow = dictionary.doc2bow(query.split())
    query_lda = lda[tfidf[query_bow]]

    sims = index[query_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    print "{:25}\t{:20}\t{}".format(comp, competencys[sims[0][0]], sims[0][1])

strategic competency     	Passion             	0.17106346786
growth potential         	Reliability         	0.352613568306
integrity                	Proactivity         	0.230119466782
intellect                	Reliability         	0.205659434199
confidence               	Proactivity         	0.333422660828
goal orientated          	Proactivity         	0.148420244455
job performance          	Creativity          	0.0
organisation             	Proactivity         	0.524779200554
creativity               	Passion             	0.0973034501076
self motivated           	Team Work           	0.287574052811
strategy                 	Employee loyalty    	0.349288731813
charisma                 	People Skills       	0.514896392822
teamwork                 	Leadership          	0.251204460859
resilient                	Passion             	0.122710265219
performance              	Creativity          	0.0
goal orientation         	Proactivity         	0.215130850673
leadership               	Proa